# Initial

In [ ]:
# import JupyterNotebooksLib as slicernb
import slicer, vtk
import os

# Set image viewer size to 50% (fill half of a cell)
slicernb.AppWindow.setWindowSize(scale=1)
# Hide patient information from slice view
slicernb.showSliceViewAnnotations(False)

# Example of installing an extension:
# slicernb.installExtensions(["SegmentEditorExtraEffects"])

# Transfer all the .tif volumes to multi-class .obj files

In [ ]:
for files in range(1,81):
    
    imagename = str(files)+'.tif'
    dataFolder = 'F:/Xinyuan/allcells-1/celllabel_255_1_579/'+imagename
    saveFolder = "D:/slicerOutput0131_1-299"
    
    # Clear scene
    slicer.mrmlScene.Clear(False)

    # Load from local file
    masterVolumeNode = slicer.util.loadVolume(dataFolder)
    outputVolumeSpacingMm = [0.000325, 0.000325, 0.001]
    masterVolumeNode.SetSpacing(outputVolumeSpacingMm)

    # Lightbox view
    slicernb.ViewLightboxDisplay("Red")

    # Create segmentation
    segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    nodename = "vol1_"+str(files)
    segmentationNode.SetName(nodename)
    slicer.modules.markups.widgetRepresentation().onRenameAllWithCurrentNameFormatPushButtonClicked()
    segmentationNode.CreateDefaultDisplayNodes() # only needed for display
    segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)

    # Create temporary segment editor to get access to effects
    segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
    segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
    segmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
    segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
    segmentEditorWidget.setSegmentationNode(segmentationNode)
    segmentEditorWidget.setMasterVolumeNode(masterVolumeNode)

    # Create segments by thresholding
    cellAll = []
    for i in range(1,300):
        cellDic = []   
        cellDic.append("cell_"+str(i))
        cellDic.append(i)
        cellDic.append(i)
        cellAll.append(cellDic)
    # segmentsFromHounsfieldUnits = [
    #     ["cell_1", 1, 1],
    #     ["cell_2", 2, 2],
    #     ["cell_3", 3, 3] ]
    segmentsFromHounsfieldUnits = cellAll

    for segmentName, thresholdMin, thresholdMax in segmentsFromHounsfieldUnits:
        # Create segment
        addedSegmentID = segmentationNode.GetSegmentation().AddEmptySegment(segmentName)
        segmentEditorNode.SetSelectedSegmentID(addedSegmentID)
        # Fill by thresholding
        segmentEditorWidget.setActiveEffectByName("Threshold")
        effect = segmentEditorWidget.activeEffect()
        effect.setParameter("MinimumThreshold",str(thresholdMin))
        effect.setParameter("MaximumThreshold",str(thresholdMax))
        effect.self().onApply()
        segmentId = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(segmentName)
#         print(segmentId)
#         print(segmentName)
        if thresholdMin < 256:
            segmentationNode.GetSegmentation().GetSegment(segmentId).SetColor(thresholdMin/255,1-thresholdMin/255,0)
        elif thresholdMin > 255 and thresholdMin < 512:
            segmentationNode.GetSegmentation().GetSegment(segmentId).SetColor(0,(thresholdMin-255)/255,1-(thresholdMin-255)/255)
        elif thresholdMin > 511 and thresholdMin < 768:
            segmentationNode.GetSegmentation().GetSegment(segmentId).SetColor(1-(thresholdMin-511)/255,0,(thresholdMin-511)/255)

    # Delete temporary segment editor
    segmentEditorWidget = None
    slicer.mrmlScene.RemoveNode(segmentEditorNode)
    segmentationNode.CreateClosedSurfaceRepresentation()
    
    if not os.path.exists(saveFolder):
        os.mkdir(saveFolder)
    saveFolder = saveFolder+'/'
    slicer.modules.segmentations.logic().ExportSegmentsClosedSurfaceRepresentationToFiles(saveFolder, segmentationNode, None,"OBJ", True, 1.0, False )
